#Loading and preparing modules

In [1]:
!pip install -qqq openai cohere tiktoken langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

In [4]:
# Langchain for chat models: without chains
def generate_response_langchain(prompt:str, system:str = "", temperature:float = 0.0, max_tokens:int = 50):

  chat = ChatOpenAI(model_name='gpt-3.5-turbo-1106', temperature=temperature, max_tokens=max_tokens, openai_api_key=openai_api_key)
  messages = [SystemMessage(content=system),
              HumanMessage(content=prompt),
              ]
  return chat(messages).content

In [5]:
prompt = "Hello there!"
response = generate_response_langchain(prompt)

print(response)

Hello! How can I assist you today?


# Zero-shot Prompting

In [6]:
## Langchain for chat models: using chains!
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)
chain = prompt | ChatOpenAI(temperature=0.7, openai_api_key=openai_api_key)
# Feel free the change test out your own product of choice, you can swap out the colorful socks in the following line with you choice of product
print(chain.invoke({"product": "colorful socks"}).content)

Rainbow Threads


Zero-shot is useful, but sometimes it might not work...




In [7]:
prompt = PromptTemplate.from_template(
    "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 2. {question}"
)
chain = prompt | ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
print(chain.invoke({"question": "True or false?"}).content)

False.


#Few-shot Prompting

In [8]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.", "output": "The answer is False."},
    {"input": "The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.", "output": "The answer is True."},
    {"input": "The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.", "output": "The answer is True."},
    {"input": "The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.", "output": "The answer is False."},
    ]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("ai", "{output}"),])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

question = "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 2. True or false"

final_prompt = ChatPromptTemplate.from_messages([
    # ("system", "You are a wondrous wizard of math."), #Optional
     few_shot_prompt, ("human", "{question}"),
    ])


In [9]:
chain = final_prompt | ChatOpenAI(temperature=0.0, openai_api_key=openai_api_key)
print(chain.invoke({"question": question}).content)

The answer is True.


#Chain-of-thought Prompting

###Zero-shot chain-of-thought

*   Note that the prompt is almost identical to the zero-shot example. The only different is to add "Let's think step by step" at the end of the prompt.



In [10]:
prompt = PromptTemplate.from_template("{question}")

question = "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 2. True or false? Let's think step by step"
chain = prompt | ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
print(chain.invoke({"question": question}).content)

Let's add up the odd numbers in the group: 15 + 5 + 13 + 7 = 40.

Since 40 is an even number, the statement "The odd numbers in this group add up to an even number" is true.


###Few-shot chain-of-thought

In [11]:
examples = [
    {"input": "The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.", "output": "Adding all the odd numbers (9, 15, 1) gives 25. The answer is False."},
    {"input": "The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.", "output": "Adding all the odd numbers (17, 19) gives 36. The answer is True."},
    {"input": "The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.", "output": "Adding all the odd numbers (11, 13) gives 24. The answer is True."},
    {"input": "The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.", "output": "Adding all the odd numbers (17, 9, 13) gives 39. The answer is False."},
    ]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("ai", "{output}"),])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

question = "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 2. True or false." #Feel free to add "let's think step by step" here

final_prompt = ChatPromptTemplate.from_messages([
    # ("system", "You are a wondrous wizard of math."), #Optional
     few_shot_prompt, ("human", "{question}"),
    ])


In [12]:
chain = final_prompt | ChatOpenAI(temperature=0.0, openai_api_key=openai_api_key)
print(chain.invoke({"question": question}).content)

Adding all the odd numbers (15, 5, 13, 7) gives 40. The answer is True.


#Self-Consistency

Let's try with a more complicated example

In [13]:
prompt = PromptTemplate.from_template(
    "{information} {question}"
)
chain = prompt | ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
responses = []

information = "When I was 6 my sister was half of my age and my mom was 5 times my age. My dad was born two years later than my mom, and my uncle and my dad are twins."
question = "Now I'm 50, how old is my uncle? Let's think step by step"

for _ in range(7):
  response = chain.invoke({"information": information, "question": question}).content
  print(f"{response}\n------------------------------------------------------------")
  responses.append(response)

prompt = PromptTemplate.from_template(
    "These are previous responses for the given question: {information} {question}: {responses}. Based on these responses, {question}. Pick the most frequent answer."
)

chain = prompt | ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
print(chain.invoke({"information": information, "question": question, "responses": responses}).content)

When you were 6, your sister was half of your age, so she was 3 years old.
At that time, your mom was 5 times your age, so she was 30 years old.
Your dad was born two years later than your mom, so he was 28 years old.
Since your dad and your uncle are twins, your uncle was also 28 years old at that time.
Now, you are 50 years old, which means 44 years have passed since you were 6.
Therefore, your uncle is now 28 + 44 = 72 years old.
------------------------------------------------------------
When you were 6, your sister was half of your age, so she was 3 years old.
At that time, your mom was 5 times your age, so she was 30 years old.
Your dad was born two years later than your mom, so he was 28 years old.
Since your dad and your uncle are twins, your uncle was also 28 years old at that time.
Now, you are 50 years old, which means 44 years have passed since you were 6.
Therefore, your uncle is now 28 + 44 = 72 years old.
------------------------------------------------------------
When

#Generate Knowledge Prompting

In [14]:
prompt = PromptTemplate.from_template("{request}")

request="Generate 4 facts about the Kermode bear, then use these facts to write a short blog post using the information? Please write the post in pure text form."
chain = prompt | ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

print(chain.invoke({"request": request}).content)

Fact 1: The Kermode bear, also known as the "spirit bear" or "ghost bear," is a rare subspecies of the American black bear.

Fact 2: The Kermode bear is primarily found in the Great Bear Rainforest of British Columbia, Canada, which is one of the largest temperate rainforests in the world.

Fact 3: What makes the Kermode bear unique is its white or cream-colored fur, which is caused by a recessive gene. Approximately 1 in 10 Kermode bears have this distinctive coat color.

Fact 4: The Kermode bear holds great cultural significance for the indigenous people of the region, who consider it a sacred and spiritual animal.

---

Blog Post: Discover the Enchanting World of the Kermode Bear

Welcome, nature enthusiasts! Today, we embark on a journey to explore the captivating realm of the Kermode bear, a remarkable creature that roams the lush forests of British Columbia, Canada. Known for its ethereal beauty and cultural significance, the Kermode bear, also referred to as the "spirit bear" or

#Tree of Thoughts

In [15]:
from IPython.core.display import Markdown
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

brainstrom_prompt = PromptTemplate.from_template(
    """Step1 :
    I have a thinking about {topic}. Could you brainstorm three distinct solutions?"""
)

evaluate_prompt = PromptTemplate.from_template(
    """Step 2:
    For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

    {solutions}"""
)

think_through_prompt = PromptTemplate.from_template(
    """Step 3:
    For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

    {evaluations}"""
)

rank_prompt = PromptTemplate.from_template(
    """Step 4:
    Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution

    {deeper_thoughts}"""
)

chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
chat_parser = chat | StrOutputParser()

# Chain #1: User input passed through as topic, the prompt runs through the llm, and the output stored as solutions.
brainstorm_solutions =  {"topic": RunnablePassthrough()} | brainstrom_prompt | {"solutions": chat_parser}

# Chain #2: The output "solutions" from brainstrom_solutions fits in the "solutions" variable in the prompt, goes through llm and the output stored as "review".
evaluate_solutions = {"solutions": brainstorm_solutions} | evaluate_prompt | {"review": chat_parser}

# Chain #3
think_through = {"evaluations": evaluate_solutions} | think_through_prompt | {"deeper_thoughts": chat_parser}

# Chain #4
rank = {"deeper_thoughts": think_through} | rank_prompt | chat

# Chain them together!
chain = ( brainstorm_solutions | evaluate_solutions | think_through | rank)

topic = "how to go to Mars"
response = chain.invoke({"topic": topic})


Markdown(response.content)


Ranking of Solutions in Order of Promise:

1. Solution 1: Genetic Modification
2. Solution 2: Underground Habitats
3. Solution 3: 3D Printing

Justification for Ranking:

1. Solution 1: Genetic Modification
This solution ranks first in promise due to its potential to significantly enhance the resilience and survival rates of humans on Mars. The ability to modify human DNA to withstand radiation, extreme temperatures, and low gravity could greatly increase the chances of long-term colonization. However, ethical concerns and public acceptance are significant obstacles that need to be addressed through transparent communication and strict regulations.

2. Solution 2: Underground Habitats
The construction of underground habitats ranks second in promise as it offers a controlled and protected environment for human habitation on Mars. These habitats can provide protection from radiation, extreme temperatures, and micrometeorites. However, ensuring structural integrity and addressing potential psychological impacts are challenges that need to be overcome through advanced construction techniques, materials, and psychological support.

3. Solution 3: 3D Printing
3D printing ranks third in promise as it has the potential to rapidly and efficiently construct structures on Mars using Martian soil as a construction material. This solution reduces reliance on Earth for materials and enables the construction of necessary structures. However, the challenges of adapting 3D printing technology to the Martian environment and ensuring the quality and durability of printed structures need to be addressed through extensive research and development.

Final Thoughts and Considerations:

- Genetic Modification: The potential benefits of genetic modification are significant, but ethical concerns and public acceptance are major obstacles. Transparent communication and strict regulations are necessary to address these concerns and ensure the ethical use of genetic engineering.

- Underground Habitats: While underground habitats offer protection from external hazards, ensuring structural integrity and addressing potential psychological impacts are crucial. Collaboration with construction companies, engineering firms, and mental health experts is necessary to overcome these challenges.

- 3D Printing: The use of 3D printing technology can revolutionize construction on Mars, but modifications to existing technology and ensuring the quality of printed structures are important considerations. Continuous research and development are needed to improve scalability and complexity, and alternative construction methods should be explored as backups.

Overall, each solution has its own promise and challenges. A combination of these solutions may provide the best approach for long-term colonization on Mars, taking into account the unique conditions and requirements of the planet.